In [1]:
import numpy as np 
import pandas as pd 
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

In [3]:
df_train = pd.read_csv('../input/novosib/train_dataset_train (3).csv')
df_test = pd.read_csv('../input/novosib/test_dataset_test.csv')

In [4]:
df_train = df_train.fillna(0)
df_test = df_test.fillna(0)

In [6]:
X = df_train.drop(["Class", "id", "Reflectance"], axis = 1)
y = df_train[["Class"]]

In [7]:
test_id = df_test['id']
X_test_ = df_test.drop(['id', "Reflectance"], axis = 1)

In [8]:
ss = StandardScaler()
X_scaled = ss.fit_transform(X)
X_t_scaled = ss.fit_transform(X_test_)

In [9]:
df_train['neighr'] = 0
neigh = RadiusNeighborsClassifier(radius = 0.034)
neigh.fit(X_scaled, y)
for i in range(1, len(X_scaled)// 200000 + 2):
    print(i)
    df_train.iloc[(i-1) * 200000:i * 200000, -1] = list(map(len, neigh.radius_neighbors(X_scaled[(i-1) * 200000:i * 200000], 0.034, return_distance = False)))

/opt/conda/lib/python3.7/site-packages/sklearn/neighbors/_classification.py:501: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._fit(X, y)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22


In [10]:
df_test['neighr'] = 0
for i in range(1, len(X_t_scaled) // 200000 + 2):
    print(i)
    df_test.iloc[(i-1) * 200000:i * 200000, -1] = list(map(len, neigh.radius_neighbors(X_t_scaled[(i-1) * 200000:i * 200000], 0.034, return_distance = False)))

1
2
3
4
5
6
7
8
9
10


In [2]:
# df_train = pd.read_csv('../input/novosib-neighr/df_train_neighr.csv') 
# df_test = pd.read_csv('../input/novosib-neighr/df_test_neighr.csv')

In [3]:
X = df_train.drop(["Class", "id"], axis = 1)
y = df_train[["Class"]]

In [4]:
test_id = df_test['id']
X_test_ = df_test.drop(['id'], axis = 1)

In [8]:
strategy = {5:402000}
oversample = SMOTE(sampling_strategy=strategy)
X, y = oversample.fit_resample(X, y)

In [1]:
from sklearn.ensemble import StackingClassifier
import xgboost as xgb
from sklearn.neighbors import KNeighborsClassifier
estimators = [('rf', RandomForestClassifier()), ('xgb', xgb.XGBClassifier()),  ('knn', KNeighborsClassifier())] 
clf = StackingClassifier(estimators = estimators, final_estimator = RandomForestClassifier(), cv = 3)#

clf.fit(X, y)

pred_clf_xbg_knn = clf.predict(X_test_) 
sub = pd.DataFrame({'id': test_id, 'Class': pred_clf_xbg_knn})
sub.to_csv('pred_clf_xbg_knn_.csv', index=False)

import os
os.chdir('./')
from IPython.display import FileLink
FileLink('pred_clf_xbg_knn.csv')

NameError: name 'RandomForestClassifier' is not defined

* clf = RandomForestClassifier(n_estimators = 100) (70/30)
 recall_score = 0.8815485209338161
* clf = RandomForestClassifier(n_estimators = 200) (70/30)
 recall_score = 0.8835150528610454
* clf = RandomForestClassifier(n_estimators = 400) (70/30)
 recall_score = 0.883010593970495
* clf = RandomForestClassifier(n_estimators = 100) (90/10)
 recall_score = 0.8928570790277804
* clf = RandomForestClassifier(n_estimators = 100) (80/20)
 recall_score = 0.8806582692105996